In [37]:
import pandas as pd
import numpy as np
from round_3_trader_v3 import BS_CALL, implied_volatility, delta
from round_3_trader_v3 import Trader
from collections import defaultdict

from datamodel import Listing, Observation, Order, OrderDepth, ProsperityEncoder, Symbol, Trade, TradingState
from typing import List, Any
import string
import jsonpickle
import numpy as np
from statistics import NormalDist
import math
import json

In [38]:
def build_order_depths(df: pd.DataFrame) -> dict:
    order_depths = defaultdict(dict)

    for _, row in df.iterrows():
        timestamp = str(row["timestamp"])
        product = row["product"]

        # Build buy_orders dictionary
        buy_orders = {}
        for i in range(1, 4):
            price = row.get(f"bid_price_{i}")
            volume = row.get(f"bid_volume_{i}")
            if pd.notna(price) and pd.notna(volume):
                buy_orders[int(price)] = int(volume)

        # Build sell_orders dictionary (volumes should be negative)
        sell_orders = {}
        for i in range(1, 4):
            price = row.get(f"ask_price_{i}")
            volume = row.get(f"ask_volume_{i}")
            if pd.notna(price) and pd.notna(volume):
                sell_orders[int(price)] = -int(volume)

        order_depths[timestamp][product] = OrderDepth(buy_orders, sell_orders)

    return dict(order_depths)

df = pd.read_csv(r'C:\Users\gonzaal\Desktop\Personal\Personal python projects\Prosperity 3\round3\data\prices_round_3_day_1.csv')

# Build the structure
order_depths_ALL = build_order_depths(df)


In [ ]:
df[df[]]

In [65]:
class Product:
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    KELP = "KELP"
    SQUID_INK = "SQUID_INK"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    DJEMBES = "DJEMBES"
    PICNIC_BASKET1 = "PICNIC_BASKET1"
    ARTIFICIAL_PICNIC_BASKET1 = "ARTIFICIAL_PICNIC_BASKET1"
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    SPREAD1 = "SPREAD1"
    VOLCANIC_ROCK = "VOLCANIC_ROCK"
    VOLCANIC_ROCK_VOUCHER_9500 = "VOLCANIC_ROCK_VOUCHER_9500"
    VOLCANIC_ROCK_VOUCHER_9750 = "VOLCANIC_ROCK_VOUCHER_9750"
    VOLCANIC_ROCK_VOUCHER_10000 = "VOLCANIC_ROCK_VOUCHER_10000"
    VOLCANIC_ROCK_VOUCHER_10250 = "VOLCANIC_ROCK_VOUCHER_10250"
    VOLCANIC_ROCK_VOUCHER_10500 = "VOLCANIC_ROCK_VOUCHER_10500"
    



BASKETS_PRODUCTS = {
    Product.PICNIC_BASKET1: 
    {
        Product.CROISSANTS : 6,
        Product.JAMS : 3,
        Product.DJEMBES : 1
    },
    Product.PICNIC_BASKET2:
    {
        Product.CROISSANTS : 4,
        Product.JAMS : 2
    }
}

BASKET1_PRODS = {
    Product.CROISSANTS : 6,
    Product.JAMS : 3,
    Product.DJEMBES : 1
}

BASKET2_PRODS = {
    Product.CROISSANTS : 4,
    Product.JAMS : 2
}


PARAMS = {
    Product.RAINFOREST_RESIN: {
        "fair_value": 10000,
        "take_width": 1,
        "clear_width": 0,
        # for making
        "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
        "join_edge": 2,  # joins orders within this edge
        "default_edge": 4,
        "soft_position_limit": 25,
    },
    Product.KELP: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": False,
        "adverse_volume": 15,
        "reversion_beta": 25,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
    Product.SQUID_INK: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": False,
        "adverse_volume": 15,
        "reversion_beta": -0.229,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
    Product.SPREAD1:{
        "spread_mean": 48.762433333333334,
        "starting_its": 30000,
        "spread_std_window": 25,
        "zscore_threshold": 11,
        "target_position": 60,
    },
    Product.VOLCANIC_ROCK_VOUCHER_9500: {
        "mean_volatility": 0.45,
        "threshold": 0.00163,
        "strike": 9500,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
     Product.VOLCANIC_ROCK_VOUCHER_9750: {
        "mean_volatility": 0.15959997370608378,
        "threshold": 0.00163,
        "strike": 9750,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
     Product.VOLCANIC_ROCK_VOUCHER_10000: {
        "mean_volatility": 0.18,
        "threshold": 0.00163,
        "strike": 10000,
        "time to maturity": 5 / 252,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
     Product.VOLCANIC_ROCK_VOUCHER_10250: {
        "mean_volatility": 0.15959997370608378,
        "threshold": 0.00163,
        "strike": 10250,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
     Product.VOLCANIC_ROCK_VOUCHER_10500: {
        "mean_volatility": 0.15959997370608378,
        "threshold": 0.00163,
        "strike": 10500,
        "time to maturity": 5 / 250,
        "std_window": 30,
        "zscore_threshold": 5.1
    },
}

Trader1 = Trader()

In [66]:
order_depths = order_depths_ALL['999000']

In [67]:
opt = Product.VOLCANIC_ROCK_VOUCHER_10250

opt_position = 0 
option_orders = Trader1.VolcanicRockCoupon_strat(order_depths = order_depths,prod=opt,position=opt_position)
option_orders

[(VOLCANIC_ROCK_VOUCHER_10250, 63, 21)]

In [68]:
Trader1.get_mid_price(order_depths[opt])

62.5

In [69]:
prd = Product.VOLCANIC_ROCK
best_bid_price = max(order_depths[prd].buy_orders.keys())
best_ask_price = min(order_depths[prd].sell_orders.keys())

Trader1.get_mid_price(order_depths[prd])

10217.0

In [70]:

best_bid_price = max(order_depths[opt].buy_orders.keys())
best_ask_price = min(order_depths[opt].sell_orders.keys())

Trader1.get_mid_price(order_depths[opt])

62.5

In [71]:
VOLCANIC_ROCK_position = 0
underlying_mid_price = Trader1.get_mid_price(order_depths[prd])
option_mid_price = Trader1.get_mid_price(order_depths[opt])

In [72]:
option_mid_price

62.5

In [73]:
K = Trader1.params[opt]['strike']
T = Trader1.params[opt]['time to maturity']
r = 0


volatility = implied_volatility(
                option_mid_price,
                underlying_mid_price,
                Trader1.params[opt]["strike"],
                T,
                r
            )

delta1 = delta(underlying_mid_price,K,T,r,volatility)

In [74]:
delta1

0.43664555921917003

In [75]:

hedge_orders = Trader1.hedge_option_orders(Product.VOLCANIC_ROCK,order_depths,option_orders,VOLCANIC_ROCK_position, opt_position,delta1)


In [76]:
hedge_orders

[(VOLCANIC_ROCK, 10216, -9)]

In [4]:
S = 10502
K1 = 9500
K2 = 9750
K3 = 10000
K4 = 10250
K5 = 10500
T = 5/252
r = 0

V1 = 99.5
V3 = 505

implied_volatility(V3,S,K3,T,r)

0.18391421773478217